# VANGUARD AB TEST


## METADATA HELP

This comprehensive set of fields will guide your analysis, helping you unravel the intricacies of client behavior and preferences.

- **client_id**: Every client’s unique ID.
- **variation**: Indicates if a client was part of the experiment.
- **visitor_id**: A unique ID for each client-device combination.
- **visit_id**: A unique ID for each web visit/session.
- **process_step**: Marks each step in the digital process.
- **date_time**: Timestamp of each web activity.
- **clnt_tenure_yr**: Represents how long the client has been with Vanguard, measured in years.
- **clnt_tenure_mnth**: Further breaks down the client’s tenure with Vanguard in months.
- **clnt_age**: Indicates the age of the client.
- **gendr**: Specifies the client’s gender.
- **num_accts**: Denotes the number of accounts the client holds with Vanguard.
- **bal**: Gives the total balance spread across all accounts for a particular client.
- **calls_6_mnth**: Records the number of times the client reached out over a call in the past six months.
- **logons_6_mnth**: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.


In [ ]:
%load_ext autoreload
%autoreload 2 

In [450]:
from cleaning import *
from mining import *
from db_handling import *
import pandas as pd
from dotenv import load_dotenv
import os
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Load environment variables
load_dotenv()

### Load Configuration

In [ ]:
# Load config.yaml
config = parse_config()

## Data Mining

In [453]:
# Creates a dictionary of all imported dataframes
dataframes = { name:import_data_from_config(config, name) for name in config['tables']}

## Data Cleaning

In [454]:
#TODO: don't impose categories?

In [455]:
# Rename columns
dataframes = rename_columns(dataframes, config)

In [456]:
# Select columns
dataframes = select_columns(dataframes, config)

In [ ]:
display_dataFrames(dataframes,'head')

In [ ]:
""" variation_visits = visits_df.merge(experiment_df, on='client_id')
display(variation_visits['variation'].value_counts())
display(variation_visits)
confirmed_steps = variation_visits[variation_visits['process_step'] == 'confirm']
display(confirmed_steps)
unique_visit_ids = confirmed_steps.drop_duplicates(subset='visit_id')
display(unique_visit_ids) """
""" visits = variation_visits.groupby(['variation','process_step']).agg({'process_step':'count'})
visits """


""" display(unique_visit_ids['variation'].value_counts()) """

### Separation

In [ ]:
client_df = dataframes['clients']
experiment_df = dataframes['experiment']
visits_df = dataframes['visits']
display(experiment_df['variation'].isna().sum())
display (client_df, experiment_df, visits_df)

In [ ]:
# drop the nulls from clients, but keep the list of the drops

nulls_client_id = client_df[client_df.isna().any(axis=1)]['client_id']
nulls_client_id

In [ ]:
client_df = client_df.dropna(axis=0)
client_df

In [ ]:
display(client_df['gender'].value_counts(dropna = False))
# x->u, keep 'U's for everything except the gender statistics

In [463]:
#client_df['gender'] = client_df['gender'].replace(to_replace=r'.*X.*', value ="U", regex=True)

In [ ]:
display(experiment_df['variation'].value_counts(dropna = False))
# keep NaN for general analysis of clients, but drop them from everywhere for test analysis

In [ ]:
# client_df, experiment_df, visit_df -> for general analysis
# new_client_df, new_experiment_df, new_visit_dfn -> for test/control analysis   experiment_df_null = 
nulls_in_experiment = experiment_df[experiment_df.isna().any(axis=1)]['client_id']
nulls_in_experiment

In [ ]:
# new df removing client ID that are null in experiment

new_experiment_df = experiment_df[~experiment_df['client_id'].isin(nulls_in_experiment)]
display(new_experiment_df.count())

new_experiment_df = new_experiment_df[~new_experiment_df['client_id'].isin(nulls_client_id)]
display(new_experiment_df.count())



In [ ]:
new_visits_df = visits_df[~visits_df['client_id'].isin(nulls_in_experiment)]
new_visits_df

In [ ]:
new_client_df = client_df[~client_df['client_id'].isin(nulls_in_experiment)]
new_client_df

### End separation

In [469]:
dataframes['clients'] = new_client_df.copy()
dataframes['experiment'] = new_experiment_df.copy()
dataframes['visits'] = new_visits_df.copy()

In [470]:
# Data Categorizing
dataframes = clean_categorical_data(dataframes, config)

In [471]:
#Convert types
dataframes = convert_types(dataframes, config)

In [ ]:
display_dataFrames(dataframes, 'head', 'dtypes', 'cat_count')

In [473]:
client_df = dataframes['clients']
experiment_df = dataframes['experiment']
visits_df = dataframes['visits']


### SQL EXPORT

In [474]:
if config['refresh_db']:

    db_password = os.getenv('SQL_PASSWORD')

    # Create database if it doesn't exist
    engine = create_db(db_password, config)

    # Export tables to database if refresh is set to true
    export_dataframes_to_sql(engine, dataframes)

    # Import data from database
    dataframes = import_all_tables_from_sql(engine)

### Local Caching

In [ ]:
""" # Save files locally in an untracked folder
export_dataframes_to_csv(dataframes) """

In [ ]:
#TODO CAREFUL DATA WONT BE PROPERLY CATEGORIZED / TYPED run after : convert_types(dataframes, config)
""" clients_df = pd.read_csv('data/cleaned/clients.csv')
experiment_df = pd.read_csv('data/cleaned/experiment.csv')
visits_df = pd.read_csv('data/cleaned/visits.csv') """

## CLEAN FRAMES

In [ ]:
display('clients :',client_df, 'experiment :',experiment_df, 'visits :',visits_df)

experiment_df['variation'].value_counts()


In [478]:
# client_since_year : redundant : drop
# client_since_month: hypothesis : the longer they are client, the more valuable to us
# client_since_month: hypothesis : the older the client is, the more valuable to us
# gender: hypothesis : the men have more balance
# number_of_accounts: hypothesis : the clients with more accounts have more balance
# calls + logons : hypothesis : active clients are more valuable to us

# process steps + time : 
    # - SUCCESS : All the steps, in order, in a reasonable amount of time for each step
    
    # - ERROR : path do not start with start : drop
    # - ERROR : path do not complete : analyse
    # - ERROR : path do not complete in order: analyse
    # - ERROR : All the steps in order but took very long
    # - ERROR : Unusual amount of time between steps

## Data Exploration

In [ ]:
# Added variation column to visits for easier analysis
visits_variations = visits_df.merge(experiment_df, on='client_id', how='right')
display(visits_variations)

In [ ]:
# Total number of visits per variation to calculate the success rate later
number_of_visits = visits_variations.groupby('variation', observed=False)['visit_id'].nunique()
display(number_of_visits)


In [ ]:
# sort by visit_id and date_time to see the process steps in order
visits_variations = visits_variations.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])
display(visits_variations)

In [ ]:
# function to filter visits with our happy path (thanks chatGPT)
def filter_visits_with_happy_path(df):
    def check_sequence(group):
        steps = list(group['process_step'])
        return steps == ['start', 'step_1', 'step_2', 'step_3', 'confirm']

    df_filtered = df.groupby('visit_id').filter(check_sequence)
    return df_filtered

happy_paths = filter_visits_with_happy_path(visits_variations)

display(happy_paths)

In [ ]:
# calculate the time difference between each step within the same visit_id in seconds
happy_paths['time_taken'] = happy_paths.groupby('visit_id')['date_time'].diff().dt.total_seconds()
happy_paths['time_taken'] = happy_paths['time_taken'].fillna(0)

display(happy_paths)

In [ ]:
# add a column to show the total time taken for each visit_id
happy_paths['total_time_taken'] = happy_paths.groupby('visit_id')['time_taken'].transform('sum')
display(happy_paths)

In [ ]:
# merge client_df with happy paths to get demographic data
happy_paths = happy_paths.merge(client_df, on='client_id', how='left')
display(happy_paths)

In [ ]:
# get the average time taken for each step
avg_time_taken = happy_paths.groupby('process_step', observed=False).agg({'time_taken':'mean'})
display(avg_time_taken)

# get the average total time taken 
avg_total_time_taken = happy_paths['total_time_taken'].mean()
display(avg_total_time_taken)

In [ ]:
# filter the happy paths without the outliers for the total time taken (1.25 times the average) 
happy_paths_filtered = happy_paths[happy_paths['total_time_taken'] <= avg_total_time_taken * 1.25]
display(happy_paths_filtered.head())

In [ ]:
# Total number of success per variation
number_of_successes = happy_paths_filtered.groupby('variation', observed=False)['visit_id'].nunique()
display(number_of_successes)


In [ ]:
# calculate the success rate per variation
success_rate = number_of_successes / number_of_visits
display(success_rate)

### correlation matrix

In [ ]:
# select only the happy paths confirmed steps
confirmed_steps = happy_paths_filtered[happy_paths_filtered['process_step'] == 'confirm']

# add numerical variation to the happy paths
confirmed_steps.loc[:, 'variation_num'] = confirmed_steps['variation'].replace({'Control': 0, 'Test': 1}).astype(int)

# add numerical gender to the happy paths
confirmed_steps.loc[:, 'gender_num'] = confirmed_steps['gender'].replace({'M': 0, 'F': 1, 'U': 2}).astype(int)

# Select numerical columns
numerical_data = confirmed_steps.select_dtypes(include=['float64', 'int64'])

# Compute the correlation matrix
correlation_matrix = numerical_data.corr()

# Display the correlation matrix
display(correlation_matrix)

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [491]:
#frequency tables

## Analysis

In [ ]:
# PROPORTION Z TEST ---- Proportion of successes is greater in Test group than in Control group
from statsmodels.stats.proportion import proportions_ztest
# H0: Proportion of complete steps in reasonable time in TEST group <= Proportion of complete steps in reasonable time in CONTROL group
# H1: Pt > Pc
alpha = 0.05
successes = [number_of_successes['Test'], number_of_successes['Control']]
just_visits = [number_of_visits['Test'], number_of_visits['Control']]
proportions_ztest(successes, just_visits, alternative = "larger")

In [ ]:
# TWO SAMPLE T TEST ---- Average total_time of success is smaller in Test group than in Control group
import scipy.stats as st
# H0: average time it took to complete steps in test group is greater or equal to one in control group 
#       (Mean_time_test>=Mean_time_control)
# H1: average time it took to complete steps is less in test group than in control group 
#       (Mean_time_test<Mean_time_control)
alpha=0.05
df_test = happy_paths_filtered[happy_paths_filtered['variation']=='Test']['total_time_taken']
df_control = happy_paths_filtered[happy_paths_filtered['variation']=='Control']['total_time_taken']
st.ttest_ind(df_test,df_control, equal_var=False, alternative = 'less')

In [ ]:
average_time_per_variation = happy_paths_filtered.groupby('variation').agg({'total_time_taken': 'mean'})
average_time_per_variation

In [ ]:
# average total time without outliers
avg_total_time = happy_paths_filtered['total_time_taken'].mean()
display(avg_total_time)

In [ ]:
# function to filter out visits with start->confirm but no happy path in between them
def filter_non_happy_path_visits(df):
    def check_non_consecutive(group):
        steps = list(group['process_step'])
        if steps[0] == 'start' and steps[-1] == 'confirm':
            if steps != ['start', 'step_1', 'step_2', 'step_3', 'confirm']:
                return True
        return False

    df_filtered = df.groupby('visit_id').filter(check_non_consecutive)
    return df_filtered

non_happy_paths = filter_non_happy_path_visits(visits_variations)

display(non_happy_paths)

In [ ]:
number_of_visits = visits_variations.groupby('variation', observed=False)['visit_id'].nunique()
display(number_of_visits)

In [ ]:
number_of_visits_non_happy = non_happy_paths.groupby('variation', observed=False)['visit_id'].nunique()
display(number_of_visits_non_happy)

In [ ]:
success_rate_non_happy = number_of_visits_non_happy / number_of_visits
display(success_rate_non_happy)
# proportion of lost people is greater in test group.

In [500]:
#TODO: consider binning / pd.cut / qcut for numerical data
#TODO: correlation matrix
#TODO: tukeys_test_outliers

In [501]:
# check back and forth between steps, lost?
# 

## Visualizations

## Conclusions